In [2]:
import pandas as pd
import numpy as np
import os

In [13]:
train_df = pd.read_csv('train.csv')
to_remove = ['LotFrontage', 'PoolQC', 'MiscFeature','Fence', 'FireplaceQu', 'Alley', 'GarageFinish','GarageCond', 'GarageCars','GarageType', 'GarageYrBlt', 'BsmtCond']
train_df.drop(to_remove, axis =1 ,inplace = True)

Deleting the first few columns based on the availability of data
Unsure of what features to drop for Garage. Garage cars vs Garage Quality.
How to deal with Garage Quality.


In [14]:
# train_df[train_df.columns[(train_df.isna().sum() / train_df.shape[0]) > 0]].shape
pd_series = train_df.isna().sum() / train_df.shape[0]
pd_series[pd_series > 0]

MasVnrType      0.005479
MasVnrArea      0.005479
BsmtQual        0.025342
BsmtExposure    0.026027
BsmtFinType1    0.025342
BsmtFinType2    0.026027
Electrical      0.000685
GarageQual      0.055479
dtype: float64

In [90]:
train_df[train_df['GarageCars'] == 0]['GarageCars'].count()
train_df[train_df['GarageArea'] == 0]['GarageArea']

39      0
48      0
78      0
88      0
89      0
       ..
1349    0
1407    0
1449    0
1450    0
1453    0
Name: GarageArea, Length: 81, dtype: int64

In [81]:


train_df['GarageType'] = train_df['GarageType'].fillna('NF')

In [91]:
# missing_vals = train_df[train_df.columns[train_df.isnull().sum() >0]]
# # list(train_df.columns)
# to_drop = list(train_df.columns[train_df.isna().sum() > 300])
# train_df.drop(to_drop, axis =1, inplace = True)

train_df[['GarageCars','GarageArea']]

,GarageCars,GarageArea
0,2,548
1,2,460
2,2,608
3,3,642
4,3,836
5,2,480
6,2,636
7,2,484
8,2,468
9,1,205


In [1]:
vars_to_use = ['MSSubClass','LotArea', 'LotFrontage', '']

train_df[['LotFrontage','LotArea']].corr()

NameError: name 'train_df' is not defined